In [3]:
from Bio import SeqIO
from datasketch import MinHash, MinHashLSH
import os


In [4]:
def split_fasta(input_fasta, output_dir, batch_size=50000):
    os.makedirs(output_dir, exist_ok=True)
    records = list(SeqIO.parse(input_fasta, "fasta"))
    total = len(records)
    print(f"Total sequences: {total}")

    batch_count = 0
    for i in range(0, total, batch_size):
        batch_records = records[i:i+batch_size]
        batch_file = os.path.join(output_dir, f"batch_{batch_count}.fasta")
        SeqIO.write(batch_records, batch_file, "fasta")
        print(f"Saved batch {batch_count} with {len(batch_records)} sequences")
        batch_count += 1

    return batch_count

In [5]:
def get_minhash(sequence, num_perm=128, ksize=3):
    mh = MinHash(num_perm=num_perm)
    for i in range(len(sequence) - ksize + 1):
        kmer = sequence[i:i+ksize]
        mh.update(kmer.encode('utf8'))
    return mh

def sequence_identity(seq1, seq2):
    length = min(len(seq1), len(seq2))
    matches = sum(a == b for a, b in zip(seq1[:length], seq2[:length]))
    return matches / length * 100

def deduplicate_batch(input_fasta, output_fasta, similarity=98.0, ksize=3, num_perm=128):
    records = list(SeqIO.parse(input_fasta, "fasta"))

    lsh = MinHashLSH(threshold=similarity/100, num_perm=num_perm)
    minhashes = {}
    id_to_record = {}

    for idx, record in enumerate(records):
        seq_str = str(record.seq)
        mh = get_minhash(seq_str, num_perm=num_perm, ksize=ksize)
        key = f"seq_{idx}"
        lsh.insert(key, mh)
        minhashes[key] = mh
        id_to_record[key] = record

    visited = set()
    representatives = []

    for key in minhashes:
        if key not in visited:
            similar_keys = lsh.query(minhashes[key])
            similar_records = [id_to_record[k] for k in similar_keys]

            group_representatives = []
            for record in similar_records:
                seq = str(record.seq)
                is_similar = False
                for kept in group_representatives:
                    kept_seq = str(kept.seq)
                    if sequence_identity(seq, kept_seq) >= similarity:
                        is_similar = True
                        break
                if not is_similar:
                    group_representatives.append(record)
            
            representatives.extend(group_representatives)
            visited.update(similar_keys)

    SeqIO.write(representatives, output_fasta, "fasta")
    print(f"Batch deduplicated: {len(representatives)} sequences saved to {output_fasta}")


In [6]:
def merge_batches(batch_dir, merged_output):
    batch_files = sorted([os.path.join(batch_dir, f) for f in os.listdir(batch_dir) if f.endswith(".fasta")])

    all_records = []
    for batch_file in batch_files:
        batch_records = list(SeqIO.parse(batch_file, "fasta"))
        all_records.extend(batch_records)

    SeqIO.write(all_records, merged_output, "fasta")
    print(f"Merged {len(batch_files)} batches into {merged_output} ({len(all_records)} sequences)")


In [ ]:
input_fasta = "output_fragments_284.fasta"
batch_dir = "batches"
merged_fasta = "merged_batches.fasta"
final_output = "final_filtered_peptides.fasta"

# 1. Split
split_fasta(input_fasta, batch_dir, batch_size=50000)

# 2. Deduplicate từng batch
batch_files = [os.path.join(batch_dir, f) for f in os.listdir(batch_dir) if f.endswith(".fasta")]
for batch_file in batch_files:
    output_batch = batch_file.replace(".fasta", "_filtered.fasta")
    deduplicate_batch(batch_file, output_batch, similarity=98.0)

# 3. Merge các batch đã lọc
merge_batches(batch_dir, merged_fasta)

# 4. Deduplicate lần cuối
deduplicate_batch(merged_fasta, final_output, similarity=98.0)
print("ALL DONE!")


Total sequences: 22358248
Saved batch 0 with 50000 sequences
Saved batch 1 with 50000 sequences
Saved batch 2 with 50000 sequences
Saved batch 3 with 50000 sequences
Saved batch 4 with 50000 sequences
Saved batch 5 with 50000 sequences
Saved batch 6 with 50000 sequences
Saved batch 7 with 50000 sequences
Saved batch 8 with 50000 sequences
Saved batch 9 with 50000 sequences
Saved batch 10 with 50000 sequences
Saved batch 11 with 50000 sequences
Saved batch 12 with 50000 sequences
Saved batch 13 with 50000 sequences
Saved batch 14 with 50000 sequences
Saved batch 15 with 50000 sequences
Saved batch 16 with 50000 sequences
Saved batch 17 with 50000 sequences
Saved batch 18 with 50000 sequences
Saved batch 19 with 50000 sequences
Saved batch 20 with 50000 sequences
Saved batch 21 with 50000 sequences
Saved batch 22 with 50000 sequences
Saved batch 23 with 50000 sequences
Saved batch 24 with 50000 sequences
Saved batch 25 with 50000 sequences
Saved batch 26 with 50000 sequences
Saved batch 